In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import re
import json
import base64

# Inicializar el driver
driver = webdriver.Firefox()

URL = ""
driver.get(URL)
time.sleep(5)

# Cerrar el botón de cookies si existe
try:
    cookies = driver.find_element(By.ID, 'sort_close')
    cookies.click()
    time.sleep(2)
except Exception as e:
    print("No se encontró el botón de cookies:", e)

# Obtener los href de los elementos con clase 'wantedItem'
wanted_items = driver.find_elements(By.CLASS_NAME, 'wantedItem')
links = [item.get_attribute("href") for item in wanted_items if item.get_attribute("href")]

lista_datos = []

# Extraer datos de cada enlace
for href in links:
    driver.get(URL + href)

    try:
        div_datos = driver.find_element(By.CLASS_NAME, "wanted_top_right")
        div_imagen = driver.find_element(By.CLASS_NAME, "wanted_top_left")

        # Funciones para extraer texto
        def extraer_texto_completo(clase):
            try:
                elemento = div_datos.find_element(By.CLASS_NAME, clase)
                items = elemento.find_elements(By.CLASS_NAME, "field__item")
                return " ".join([item.text.strip() for item in items if item.text.strip()])
            except:
                return ""

        def extraer_texto(clase):
            try:
                return div_datos.find_element(By.CLASS_NAME, clase).text.strip()
            except:
                return ""

        def extraer_imagen(clase):
            try:
                imagen_elemento = div_imagen.find_element(By.CLASS_NAME, clase).find_element(By.TAG_NAME, "img")
                print("a")
                return imagen_elemento.get_attribute("src")
            except:
                return ""

        # Extraer datos
        person_data = {
            "Nombre": extraer_texto("field--name-field-w-first-name"),
            "Alias": extraer_texto_completo("field--node-field-alias"),
            "Pais": extraer_texto("field--name-field-enfast-country"),
            "Crimen": extraer_texto_completo("field--name-field-crime"),
            "Sexo": extraer_texto_completo("field--name-field-gender"),
            "Altura": extraer_texto_completo("field--name-field-approximate-height"),
            "Color_ojos": extraer_texto_completo("field--name-field-eye-colour"),
            "Identificadores": extraer_texto_completo("field--node-field-characteristics"),
            "Fecha_nac": extraer_texto_completo("field--name-field-date-of-birth"),
            "Nacionalidad": extraer_texto_completo("field--name-field-nationality"),
            "Etnia": extraer_texto_completo("field--name-field-ethnic-origin"),
            "Idiomas": extraer_texto_completo("field--node-field-languages"),
            "Estado_caso": extraer_texto_completo("field--name-field-state-of-case"),
            "Fecha_publicacion": extraer_texto_completo("field--name-field-w-last-name"),
            "Imagen": extraer_imagen("field__item show")
        }

        lista_datos.append({"persona": person_data})

    except Exception as e:
        print(f"Error al extraer datos de {href}:", e)

# Crear el JSON con la estructura solicitada
data = {"personas": lista_datos}

with open('masbuscados.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

driver.close()
